In [1]:
!pip install -q -U keras-tuner

In [2]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
(img_train,label_train),(img_test,label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
img_train = img_train.astype("float32")/255.0

img_test = img_test.astype("float32")/255.0


In [5]:
def build_model(hp):
  n_hidden = hp.Int("n_hidden",min_value=0,max_value=10,default=2)
  n_neurons = hp.Int("n_neurons",min_value=16,max_value=256)
  learning_rate = hp.Float("learning_rate",min_value=1e-4,max_value=1e-2,sampling="log")
  optimizer = hp.Choice("optimizer",values=["sgd","adam"])
    
  if optimizer == "sgd":
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  else:
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  model = keras.models.Sequential()
  model.add(keras.layers.Flatten())
  for _ in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons,activation="relu"))
  model.add(keras.layers.Dense(10,activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
  return model

In [6]:
random_search_tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    overwrite=True,
    directory="my_dir",
    project_name="my_rnd_search",
)

In [7]:
random_search_tuner.search(img_train,label_train,epochs=10,validation_split=0.2)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.890333354473114

Best val_accuracy So Far: 0.890500009059906
Total elapsed time: 00h 01m 16s


In [8]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [10]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
best_params = top3_params[0].values

In [11]:
best_params

{'n_hidden': 6,
 'n_neurons': 97,
 'learning_rate': 0.0007947409954216574,
 'optimizer': 'adam'}

In [15]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 6
n_neurons: 97
learning_rate: 0.0007947409954216574
optimizer: adam
Score: 0.890500009059906


In [18]:
best_model.fit(img_train,label_train,epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 935us/step - accuracy: 0.9027 - loss: 0.2671
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 932us/step - accuracy: 0.9057 - loss: 0.2551
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9099 - loss: 0.2436
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 878us/step - accuracy: 0.9117 - loss: 0.2341
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 929us/step - accuracy: 0.9134 - loss: 0.2301


In [19]:
test_loss,test_accuarcy = best_model.evaluate(img_test,
                                              label_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - accuracy: 0.8801 - loss: 0.3562


In [21]:
test_loss,test_accuarcy

(0.34018340706825256, 0.8801000118255615)